In [ ]:
import serial
import time
from flask import Flask, request
import pymysql
import threading
from datetime import datetime

ser = serial.Serial('COM3', 9600)

app = Flask(__name__)

def db_open():
    host = '43.201.161.172' 
    user = 'test' 
    password = 'test'
    database = 'mydb'
    # global site_id 
    db = pymysql.connect(host= host, user =user, password =password, database=database)
    return db

def update_db(data):
      
    if data  == 'led_on':
        sw = 'light_sensor'
        value = 1
    elif data == 'led_off':
        sw = 'light_sensor'
        value = 0
    elif data == 'fan_on':
        sw = 'fan'
        value = 1
    elif data == 'fan_off':
        sw = 'fan'
        value = 0
    elif data == 'open':
        sw = 'sub_motor'
        value = 1
    elif data == 'close':
        sw = 'sub_motor'
        value = 0

    site_id = '02'

    db = db_open()
    curs = db.cursor()


    sql = f'''SELECT fan, sub_motor, light_sensor from mydb.internal 
                WHERE sites_site_id = {site_id}
                ORDER BY sites_site_id DESC, in_date DESC
                LIMIT 1'''
    curs.execute(sql)
    data = curs.fetchone()
    in_date = data[1]
    in_time = data[2]
    if data[3] != value :
        sql = f'''UPDATE mydb.internal SET {sw}= {value} 
                 WHERE sites_site_id = {site_id} AND in_date = {in_date} AND in_time = {in_time}'''
        curs.execute(sql)
        db.commit()

def internal_db():
    site_id = '02'
    # Receive sensor data from Arduino.
    line = ser.readline().decode('utf-8').rstrip() # Read data sent over serial communication.
    data = line.split(',') # Save the data as a comma-separated list
    # Check if the data is valid.
    if len(data) == 4:

        # Save the data to the database.
        temperature = data[0].split("=")[1]
        humidity = data[1].split("=")[1]
        mapsoil = data[2].split("=")[1]
        co2 = data[3].split("=")[1]

        db = db_open()
        curs = db.cursor()
       
        now = datetime.now() # 현재 시각 데이터 생성
        base_date = now.strftime('%Y%m%d') # 현재 날짜 YYMMDD 형식
        base_time = now.strftime('%H%M')[:-1] + '0' # 10분단위

        sql = f'''SELECT fan, sub_motor, light_sensor from mydb.internal 
                  WHERE sites_site_id = {site_id}
                  ORDER BY sites_site_id DESC, in_date DESC
                  LIMIT 1'''
        curs.execute(sql)
        data = curs.fetchone()

        fan = data[0]
        door = data[1]
        led = data[2]
        temperature = float(temperature)
        humidity =float(humidity)
        co2 = float(co2)
        mapsoil = float(mapsoil)

        sql = '''insert ignore into mydb.internal 
                        (sites_site_id,in_date, in_time, temperature, humidity, CO2, soil_humidity, fan, sub_motor, light_sensor)
                 values (%s, %s, %s, %s, %s, %s, %s, %s , %s, %s)'''

        values = (site_id, base_date, base_time, temperature, humidity, co2, mapsoil, fan, door, led)

        # sql = '''insert ignore into mydb.internal 
        #         (sites_site_id,in_date, in_time, temperature, humidity, CO2, soil_humidity)
        # values (%s, %s, %s, %s, %s, %s, %s)'''

        # values = (site_id, base_date, base_time, temperature, humidity, co2, mapsoil)
        curs.execute(sql, values)
        db.commit()
        db.close()

def db_thread():
    while True :
        internal_db()
        time.sleep(600)

def fan_worker_thread(status):
    # 팬 제어 작업 수행
    # status 값에 따라 팬을 켜거나 끄는 작업을 수행
    ser.write(status.encode())
    return 200
def servo_worker_thread(status):
    # 서보 제어 작업 수행
    # status 값에 따라 서보를 움직이는 작업을 수행
    ser.write(status.encode())
    return 200
def led_worker_thread(status):
    # LED 제어 작업 수행
    # status 값에 따라 LED를 켜거나 끄는 작업을 수행
    ser.write(status.encode())
    return 200


@app.before_first_request
def activate_background_task():
    thread = threading.Thread(target=db_thread)
    thread.start()


@app.route('/FARM/<user_id>/<site_id>/CONTROL/ETC/ETC_1', methods=['GET','POST'])
def fan_on(user_id, site_id):
    if request.method== 'POST':
        data = request.get_json()['status'] #90(on) or 180(off) 
        fan_thread = threading.Thread(target=fan_worker_thread, args=(data,))
        fan_thread.start()
        update_db(data)
        return '성공', 200
    elif request.method == 'GET':
        return "GET", 203

    

@app.route('/FARM/<user_id>/<site_id>/CONTROL/ETC/ETC_2', methods=['GET','POST'])
def servo_on(user_id, site_id):
    if request.method== 'POST':
        data = request.get_json()['status'] #90(on) or 180(off)
        servo_thread = threading.Thread(target=servo_worker_thread, args=(data,))
        servo_thread.start()
        update_db(data)
        return '성공', 200
    elif request.method == 'GET':
        return "GET", 203

@app.route('/FARM/<user_id>/<site_id>/CONTROL/ETC/ETC_3', methods=['GET','POST'])
def LED_on(user_id, site_id):

    if request.method== 'POST':
        data = request.get_json()['status'] #90(on) or 180(off)
        LED_thread = threading.Thread(target=fan_worker_thread, args=(data,))
        LED_thread.start()
        update_db(data)
        return '성공', 200
    elif request.method == 'GET':
        return "GET", 203


def db_init():
    site_id = '02'
    # Receive sensor data from Arduino.
    line = ser.readline().decode('utf-8').rstrip() # Read data sent over serial communication.
    data = line.split(',') # Save the data as a comma-separated list
    for i in data :
        print(i)
    # Check if the data is valid.
    if len(data) == 4:

        # Save the data to the database.
        temperature = data[0].split("=")[1]
        humidity = data[1].split("=")[1]
        mapsoil = data[2].split("=")[1]
        co2 = data[3].split("=")[1]

        db = db_open()
        curs = db.cursor()
       
        now = datetime.now() # 현재 시각 데이터 생성
        base_date = now.strftime('%Y%m%d') # 현재 날짜 YYMMDD 형식
        base_time = now.strftime('%H%M')

        fan = 0
        door = 0
        led = 0
        temperature = float(temperature)
        humidity =float(humidity)
        co2 = float(co2)
        mapsoil = float(mapsoil)

        sql = '''insert ignore into mydb.internal 
                        (sites_site_id,in_date, in_time, temperature, humidity, CO2, soil_humidity, fan, sub_motor, light_sensor)
                 values (%s, %s, %s, %s, %s, %s, %s, %s , %s, %s)'''

        values = (site_id, base_date, base_time, temperature, humidity, co2, mapsoil, fan, door, led)

        curs.execute(sql, values)
        db.commit()
        db.close()
    

if __name__ == '__main__':
    db_init()
    app.run(host="172.16.1.250", port=5000)



In [ ]:
ser.close()